In [1]:
import os
import tensorflow as tf
from tensorflow.core.util import event_pb2
from tensorflow.python.lib.io import tf_record

2024-03-20 10:02:48.822437: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-20 10:02:48.825453: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-20 10:02:48.873392: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-20 10:02:49.567478: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
logdir = "logs/"
models_files = [x for x in os.listdir(logdir) if "no_metadata" in x]
models_files

['end-to-end-no_metadata-ncb_1-ks_7-nf_32-fvs_30-lr_0.001-mlp_hl_2-mlp_hu_64_20240314_203522',
 'end-to-end-no_metadata-ncb_2-ks_9-nf_64-fvs_10-lr_0.001-mlp_hl_2-mlp_hu_64_20240317_140032',
 'end-to-end-no_metadata-ncb_1-ks_5-nf_16-fvs_10-lr_0.001-mlp_hl_2-mlp_hu_64_20240314_191905',
 'end-to-end-no_metadata-ncb_2-ks_9-nf_128-fvs_10-lr_0.001-mlp_hl_2-mlp_hu_64_20240317_145129',
 'end-to-end-no_metadata-ncb_2-ks_7-nf_64-fvs_20-lr_0.001-mlp_hl_2-mlp_hu_64_20240316_125443',
 'end-to-end-no_metadata-ncb_3-ks_9-nf_128-fvs_30-lr_0.001-mlp_hl_2-mlp_hu_64_20240317_221319',
 'end-to-end-no_metadata-ncb_3-ks_9-nf_32-fvs_30-lr_0.001-mlp_hl_2-mlp_hu_64_20240316_235856',
 'end-to-end-no_metadata-ncb_2-ks_9-nf_64-fvs_10-lr_0.001-mlp_hl_2-mlp_hu_64_20240316_140724',
 'end-to-end-no_metadata-ncb_3-ks_9-nf_16-fvs_10-lr_0.001-mlp_hl_2-mlp_hu_64_20240316_233004',
 'end-to-end-no_metadata-ncb_2-ks_9-nf_16-fvs_30-lr_0.001-mlp_hl_2-mlp_hu_64_20240316_135446',
 'end-to-end-no_metadata-ncb_2-ks_9-nf_32-fvs_20

In [3]:
mappings = {"Training": "train_loss", "Validation": "val_loss", "Accuracy": "val_acc"}

In [4]:
def extract_event_values(file_path):
    values = []
    for record in tf_record.tf_record_iterator(file_path):
        event = event_pb2.Event()
        event.ParseFromString(record)
        for value in event.summary.value:
            values.append(value.simple_value)
    return values

In [5]:
def extract_model_data(model_name):
    subdirs = [x for x in os.listdir(logdir + model_name) if "events" not in x]
    model_vals = {}
    for _, key in mappings.items():
        model_vals[key] = []

    for subdir in subdirs:
        events_file = [
            x for x in os.listdir(logdir + model_name + "/" + subdir) if "events" in x
        ]
        model_value_to_add = mappings[subdir.split("_")[-1]]
        record_path = logdir + model_name + "/" + subdir + "/" + events_file[0]
        values = extract_event_values(record_path)
        model_vals[model_value_to_add] = values
    return model_vals

In [6]:
print(f"len of models_files: {len(models_files)}")

len of models_files: 247


In [7]:
# For each model file check if they have more than 3 subdirs
total_models_with_data = 0
for model in models_files:
    subdirs = [x for x in os.listdir(logdir + model)]
    if len(subdirs) > 3:
        total_models_with_data += 1

total_models_with_data

226

In [8]:
models_data = {}
for model_name in models_files:
    # for each model, get the subdirectories
    model_save_name = model_name.split("end-to-end-no_metadata-")[-1][:-16]
    model_data = extract_model_data(model_name)
    models_data[model_save_name] = model_data

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


In [9]:
import pandas as pd

In [10]:
# Create a dataframe with the data
df = pd.DataFrame(models_data).T

In [11]:
df

,train_loss,val_loss,val_acc
ncb_1-ks_7-nf_32-fvs_30-lr_0.001-mlp_hl_2-mlp_hu_64,"[0.02792280912399292, 0.03349711000919342, 0.0...","[0.027408068999648094, 0.022538185119628906, 0...","[0.9897348284721375, 0.9936698079109192, 0.993..."
ncb_2-ks_9-nf_64-fvs_10-lr_0.001-mlp_hl_2-mlp_hu_64,"[0.03074588067829609, 0.05020670220255852, 0.0...","[0.040046993643045425, 0.025761963799595833, 0...","[0.9924721717834473, 0.991787850856781, 0.9931..."
ncb_1-ks_5-nf_16-fvs_10-lr_0.001-mlp_hl_2-mlp_hu_64,[],[],[]
ncb_2-ks_9-nf_128-fvs_10-lr_0.001-mlp_hl_2-mlp_hu_64,"[0.040109604597091675, 0.04194993898272514, 0....","[0.025668740272521973, 0.062107812613248825, 0...","[0.9921300411224365, 0.9928143620491028, 0.992..."
ncb_2-ks_7-nf_64-fvs_20-lr_0.001-mlp_hl_2-mlp_hu_64,[],[],[]
...,...,...,...
ncb_4-ks_9-nf_32-fvs_10-lr_0.001-mlp_hl_2-mlp_hu_64,"[0.6934837102890015, 0.6941685676574707, 0.693...","[0.693891167640686, 0.6932921409606934, 0.6930...","[0.493242084980011, 0.493242084980011, 0.50675..."
ncb_1-ks_5-nf_64-fvs_30-lr_0.001-mlp_hl_2-mlp_hu_64,"[0.040004465728998184, 0.039493195712566376, 0...","[0.02331765741109848, 0.019928211346268654, 0....","[0.9929854869842529, 0.9924721717834473, 0.988..."
ncb_3-ks_9-nf_64-fvs_10-lr_0.001-mlp_hl_2-mlp_hu_64,"[0.1344611644744873, 0.13543838262557983, 0.08...","[0.04698236659169197, 0.02300698310136795, 0.2...","[0.9873396158218384, 0.9931565523147583, 0.920..."
ncb_1-ks_3-nf_16-fvs_10-lr_0.0001-mlp_hl_2-mlp_hu_32,"[0.07144593447446823, 0.05390444025397301, 0.0...","[0.08058194816112518, 0.057717397809028625, 0....","[0.9714285731315613, 0.9840889573097229, 0.988..."


In [12]:
# Remove rows with train_loss = []
df = df[df["train_loss"].apply(lambda x: len(x) > 0)]
df

,train_loss,val_loss,val_acc
ncb_1-ks_7-nf_32-fvs_30-lr_0.001-mlp_hl_2-mlp_hu_64,"[0.02792280912399292, 0.03349711000919342, 0.0...","[0.027408068999648094, 0.022538185119628906, 0...","[0.9897348284721375, 0.9936698079109192, 0.993..."
ncb_2-ks_9-nf_64-fvs_10-lr_0.001-mlp_hl_2-mlp_hu_64,"[0.03074588067829609, 0.05020670220255852, 0.0...","[0.040046993643045425, 0.025761963799595833, 0...","[0.9924721717834473, 0.991787850856781, 0.9931..."
ncb_2-ks_9-nf_128-fvs_10-lr_0.001-mlp_hl_2-mlp_hu_64,"[0.040109604597091675, 0.04194993898272514, 0....","[0.025668740272521973, 0.062107812613248825, 0...","[0.9921300411224365, 0.9928143620491028, 0.992..."
ncb_3-ks_9-nf_128-fvs_30-lr_0.001-mlp_hl_2-mlp_hu_64,"[0.049478188157081604, 0.027880514040589333, 0...","[0.10479486733675003, 0.032753508538007736, 0....","[0.9801539778709412, 0.9907613396644592, 0.991..."
ncb_3-ks_9-nf_32-fvs_30-lr_0.001-mlp_hl_2-mlp_hu_64,"[0.058998532593250275, 0.03952853009104729, 0....","[0.034551188349723816, 0.04436846077442169, 0....","[0.9885371923446655, 0.9857998490333557, 0.990..."
...,...,...,...
ncb_4-ks_9-nf_32-fvs_10-lr_0.001-mlp_hl_2-mlp_hu_64,"[0.6934837102890015, 0.6941685676574707, 0.693...","[0.693891167640686, 0.6932921409606934, 0.6930...","[0.493242084980011, 0.493242084980011, 0.50675..."
ncb_1-ks_5-nf_64-fvs_30-lr_0.001-mlp_hl_2-mlp_hu_64,"[0.040004465728998184, 0.039493195712566376, 0...","[0.02331765741109848, 0.019928211346268654, 0....","[0.9929854869842529, 0.9924721717834473, 0.988..."
ncb_3-ks_9-nf_64-fvs_10-lr_0.001-mlp_hl_2-mlp_hu_64,"[0.1344611644744873, 0.13543838262557983, 0.08...","[0.04698236659169197, 0.02300698310136795, 0.2...","[0.9873396158218384, 0.9931565523147583, 0.920..."
ncb_1-ks_3-nf_16-fvs_10-lr_0.0001-mlp_hl_2-mlp_hu_32,"[0.07144593447446823, 0.05390444025397301, 0.0...","[0.08058194816112518, 0.057717397809028625, 0....","[0.9714285731315613, 0.9840889573097229, 0.988..."


In [13]:
df.to_csv("models_data.csv")